In [ ]:
from mmengine.runner import Runner
from torchvision.datasets import CIFAR10
from mmengine.model import BaseModel

runner=Runner(train_dataloader=dict(
        model=BaseModel(),
        batch_size=32,
        sampler=dict(
            type='DefaultSampler',
            shuffle=True,
        ),
        dataset=CIFAR10('./data/cifar10/'),
        collate_fn=dict(type='default_collate'),
        word_dir='./work_dir/'
))

1. sampler是mmengine里构造dataloader必须传入的参数，默认的DefaultSampler是采样器，用来从数据集中采取哪些样本组成
一个批次
2. collate_fn是用来将获取到的一个批次的样本转换为tensor，并且将数据按照batch维度拼接

In [ ]:
from mmengine.registry import DATASETS,TRANSFORMS
from mmengine.dataset.base_dataset import Compose
import torchvision.transforms as tvt
from torchvision.datasets import CIFAR10

#注册数据集，预处理过程一并构建
@DATASETS.register_module(name='Cifar10',force=False)
def build_cifar10_transforms(transforms=None,**kwargs):
    if isinstance(transforms,dict):
        transforms=[transforms]
    elif isinstance(transforms,(list,tuple)):
        transforms=Compose(transforms)
    return CIFAR10(**kwargs,transform=transforms)

#注册torchvision中的transforms
TRANSFORMS.register_module('RandomCrop',module=tvt.RandomCrop)
TRANSFORMS.register_module('RandomHorizontalFlip',module=tvt.RandomHorizontalFlip)
TRANSFORMS.register_module('ToTensor',module=tvt.ToTensor),
TRANSFORMS.register_module('Normalize',module=tvt.Normalize)

from mmengine.model import BaseModel

class MyModel(BaseModel):
    def _init__(self):
        super().__init__()
    
    def forward(self,x):
        pass
from mmengine.runner import Runner
runner=Runner(
    model=MyModel(),
    work_dir='./work_dirs',
    train_dataloader=dict(
        batch_size=32,
        sampler=dict(type="DefaultSampler",shuffle=True,
                     collate_fn=dict(type='default_collate')),
        dataset=dict(
            type='Cifar10',
            root='./data/cifar10/',
            train=True,
            download=True,
            transform=[
                dict(type='RandomCrop',size=32,padding=4),
                dict(type='RandomHorizontalFlip'),
                dict(type='ToTensor'),
                dict(type='Normalize',mean=[0.4914, 0.4822, 0.4465],std=[0.2023, 0.1994, 0.2010]),
            ]
        )
    ))

